In [1]:
!pip uninstall -y protobuf tensorflow

!pip install protobuf==3.20.3 tensorflow

Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 2.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
grain 0.2.15 requires protobuf>=5.28.3, but you have protobuf 3.20.3 which is incompatible.
onnx 1.20.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.utils import image_dataset_from_directory, load_img, img_to_array
import zipfile
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2026-01-16 13:18:08.791315: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768569488.813216      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768569488.819826      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768569488.837099      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768569488.837125      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768569488.837127      55 computation_placer.cc:177] computation placer alr

In [3]:
#setting up important var for loading images later on
BATCH_SIZE=32
IMG_SIZE=(224,224)
SEED=42
EXTRACT_PATH="/kaggle/input/civic-issue-dataset/Dataset"
selected_class=["NoIssue","Issues"]

In [4]:
#loading up images and dividing them for the neural net
train_data=tf.keras.utils.image_dataset_from_directory(
    EXTRACT_PATH,
    class_names=selected_class,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary" 
)

#validation dataset
validation_data=tf.keras.utils.image_dataset_from_directory(
    EXTRACT_PATH,
    class_names=selected_class,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary"
)
#now dividing the validation data set into 
total_number_of_batches_in_validation_data=validation_data.cardinality().numpy() #converts total number of batches in the set and converts the tensor into python integer with .numpy function here
no_of_batches_in_validation_data=total_number_of_batches_in_validation_data//2 #this is a floor division operator


#now from the set of batches of the image creating subset into the validation and the test subset
validation=validation_data.take(no_of_batches_in_validation_data)
test=validation_data.skip(no_of_batches_in_validation_data)

Found 19549 files belonging to 2 classes.
Using 15640 files for training.


I0000 00:00:1768569548.137231      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1768569548.141079      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 19549 files belonging to 2 classes.
Using 3909 files for validation.


In [ ]:
#loading up images and dividing them for the neural net
train_data=tf.keras.utils.image_dataset_from_directory(
    EXTRACT_PATH,
    class_names=selected_class,
    validation_split=0.4,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary" 
)

#validation dataset
validation_data=tf.keras.utils.image_dataset_from_directory(
    EXTRACT_PATH,
    class_names=selected_class,
    validation_split=0.4,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary"
)
#now dividing the validation data set into 
total_number_of_batches_in_validation_data=validation_data.cardinality().numpy() #converts total number of batches in the set and converts the tensor into python integer with .numpy function here
no_of_batches_in_validation_data=total_number_of_batches_in_validation_data//2 #this is a floor division operator


#now from the set of batches of the image creating subset into the validation and the test subset
validation=validation_data.take(no_of_batches_in_validation_data)
test=validation_data.skip(no_of_batches_in_validation_data)

In [5]:
print(f"Class names: {train_data.class_names}")
print("For values:\n")
for i, class_name in enumerate(train_data.class_names):
    print(f"{class_name}:{i}\n")

Class names: ['NoIssue', 'Issues']
For values:

NoIssue:0

Issues:1



In [6]:
#to augment data which we should do while training 
data_augmentation=tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.15),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
    tf.keras.layers.GaussianNoise(0.05),
    tf.keras.layers.RandomTranslation(0.1,0.1),
    tf.keras.layers.RandomCrop(224,224),
    tf.keras.layers.RandomBrightness(factor=0.2),
    tf.keras.layers.RandomSaturation(0.05),
    tf.keras.layers.RandomFlip("vertical"),
])

#setting var to send for training to send info about input
input=tf.keras.Input(shape=(224,224,3))


#now the acutal model which would be we want conv2d->pooling->conv2d->pooling->conv2d_>pooling-> flatten all the data two dense layer with relu
model=tf.keras.Sequential([
    input,
    data_augmentation,
    tf.keras.layers.Rescaling(1./255),

#CNN LAYERS
    tf.keras.layers.Conv2D(32,3,padding='same',activation="relu"),
    #here 32 is the number of filters/kernels, padding is same so it extends for the edges with 0 values and activation we chose is relu
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64,3, padding="same",activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(128,3,padding="same",activation="relu"),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(256,3,padding="same",activation="relu"),
    tf.keras.layers.MaxPooling2D(),

    #now flatten this cube of input
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5), #this makes the neural net to ignore 50 percent of the units all of the time so it would not look for same type of data everytime and prevent overfitting
    tf.keras.layers.Dense(1,activation="sigmoid")
])
#now compile and fit them
model.compile(optimizer="adam",
             loss=BinaryCrossentropy(from_logits=False),
             metrics=["accuracy",
             tf.keras.metrics.Precision(name="Precision"),
             tf.keras.metrics.Recall(name="recall"),])
history=model.fit(
    train_data,
    validation_data=validation,
    epochs=100,
    callbacks=[
        EarlyStopping(
            monitor="val_accuracy", #checks for val_accuracy
            patience=5,#wait tills 5 epochs
            restore_best_weights=True,#uses best weight
        ),
        ModelCheckpoint(
            "best_model.keras",#givesbest model according to val_accuracy
            monitor="val_accuracy",
            save_best_only=True,
            verbose=1 #prints only certain line of epoch for 1 and for 0 is silence and for 2 is every line
        )
    ]
)


test_loss,test_acc,test_precision,test_recall=model.evaluate(validation_data)
print(f"Accuracy: {test_acc:4f}, Precision:{test_precision:.4f}, Recall:{test_recall:.4f}")

Epoch 1/100


I0000 00:00:1768569565.517425     139 cuda_dnn.cc:529] Loaded cuDNN version 91002


489/489 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - Precision: 0.8026 - accuracy: 0.7994 - loss: 0.5399 - recall: 0.9949
Epoch 1: val_accuracy improved from -inf to 0.80072, saving model to best_model.keras
489/489 ━━━━━━━━━━━━━━━━━━━━ 94s 174ms/step - Precision: 0.8026 - accuracy: 0.7994 - loss: 0.5399 - recall: 0.9949 - val_Precision: 0.8011 - val_accuracy: 0.8007 - val_loss: 0.4176 - val_recall: 0.9994
Epoch 2/100
488/489 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - Precision: 0.8063 - accuracy: 0.8045 - loss: 0.4434 - recall: 0.9958
Epoch 2: val_accuracy improved from 0.80072 to 0.80123, saving model to best_model.keras
489/489 ━━━━━━━━━━━━━━━━━━━━ 58s 118ms/step - Precision: 0.8063 - accuracy: 0.8045 - loss: 0.4433 - recall: 0.9958 - val_Precision: 0.8027 - val_accuracy: 0.8012 - val_loss: 0.3930 - val_recall: 0.9974
Epoch 3/100
488/489 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - Precision: 0.8138 - accuracy: 0.8102 - loss: 0.4126 - recall: 0.9899
Epoch 3: val_accuracy improved from 0.80123 to 0.89242,

For accuracy:
1) We have total of 15.6k  images of issues and 3954 images of non-issues and total of 19549 images.
2) For threshold accuracy we take a very dumb modal that would always predict the class with higher images so which would be:
       threshold_accuracy=(15640/19549)*100=80.004%
3) However, we do need to figure out recall and precision values which we take harmonic mean of must be >=0.65
   NOTE: Why do we take recall and precision here?
   => We do it because we have unbalanced data not only we need accurate value we need values with good precision and recall value where,<br>
   precision=(True positive values(values we predicted positive and is positive))/(No of True values we predict(both false and true positive))<br>
   recall=(true +ve(no of predicted=actual true)/Total actual positive value(both true positive we predict and false negative we predict))

## For training exp1:
1) We have validation of 20% data shuffled
2) 4 layers of cnn
   All the metrics have:
   a) accuracy=96.07
   b) precision=97.26
   c) recall=97.82

## For tranining exp2:
1) We have validation of 40% data shuffled
2) 4 layers of cnn
   All the metrics have:
   a) accuracy=
   b) precision=
   c) recall=


In [ ]:
os.makedirs("/kaggle/working/models",exist_ok=True)
model.save("/kaggle/working/models/pothole_model.keras")

In [7]:
result=model.evaluate(test)
print(f"For the unseen test data of the entire training this model has accuracy of {result[1]:.4f}, precision of {result[2]:.4f}, and recall of {result[3]:.4f}")

62/62 ━━━━━━━━━━━━━━━━━━━━ 12s 94ms/step - Precision: 0.9742 - accuracy: 0.9676 - loss: 0.1100 - recall: 0.9853
For the unseen test data of the entire training this model has accuracy of 0.9607, precision of 0.9726, and recall of 0.9782


In [9]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     6,422,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,433,605 (77.95 MB)

 Trainable params: 6,811,201 (25.98 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 13,622,404 (51.97 MB)

Harmonic mean we got is 0.95855 which is a very good value for the data

In [ ]:
def predict_for_single_image(model, img_path):
    img=tf.keras.utils.load_img(img_path, target_size=(224,224))
    img_array=tf.keras.utils.img_to_array(img)
    img_array=np.expand_dims(img_array, axis=0)
    prediction=model.predict(img_array)
    probability=prediction[0][0]
    if probability>0.5:
        return 1;
    return 0;

In [ ]:
#to load up images
print(f"For i1 we got {predict_for_single_image(model, "/kaggle/input/helpppp/helppp/i1.png")}")
print(f"For i11 we got {predict_for_single_image(model, "/kaggle/input/helpppp/helppp/i11.jpeg")}")
print(f"For i2 we got {predict_for_single_image(model, "/kaggle/input/helpppp/helppp/i2.png")}")
print(f"For i3 we got {predict_for_single_image(model, "/kaggle/input/helpppp/helppp/i3.png")}")
print(f"For n1 we got {predict_for_single_image(model, "/kaggle/input/helpppp/helppp/n1.jpg")}")
print(f"For n2 we got {predict_for_single_image(model, "/kaggle/input/helpppp/helppp/n2.jpg")}")
print(f"For n3 we got {predict_for_single_image(model, "/kaggle/input/helpppp/helppp/n3.png")}")
print(f"For n4 we got {predict_for_single_image(model, "/kaggle/input/helpppp/helppp/n4.jpeg")}")


In [ ]:
import shutil

# Zip the folder for easy download
shutil.make_archive("/kaggle/working/models", 'zip', "/kaggle/working/models")

In [ ]:
#this is to see what our model did and how it is predicting
def visualisation(model,dataset,num_images=32):
    images, label=next(iter(dataset.take(1))) #take 1 takes one batch from tensorflow data set
    #where (image, 0 or 1) and this is returned as ptr or itr and next just gives access to it
    #following model is trained model as above
    pred=model.predict(images)
    plt.figure(figsize=(90,80))#image with 15 inches height and 10 inches width

    #randomly taking images
    indices=np.random.choice(range(len(images)),num_images,replace=False)
    #this randomly takes images from total number of images with replace=false so no duplicate images will be passes
    

    for i, idx in enumerate(indices):
        ax=plt.subplot(8,4,i+1) #creating subplot of 8*4 grid
        #converting images into the original scale as we rescale it into the image
        plt.imshow(images[idx].numpy().astype("uint8"))#displays image by converting first tensor into pixel values and then uint8 ie 0-255 
        actual_val=int(label[idx])
        predicted_val=1 if pred[idx]>0.5 else 0
        color ="green" if actual_val==predicted_val else "red"
        title=f"Actual:{actual_val}\nPredicted:{predicted_val}"
        plt.title(title,color=color,fontsize=40,fontweight="bold")
        plt.axis("off")

    #adding horizontal and vertical spaces between plots
    plt.subplots_adjust(hspace=0.6,wspace=0.3)
    plt.tight_layout
    plt.show()

In [ ]:
#seeing how model predicted for different images
visualisation(model,train_data)

In [ ]:
visualisation(model,validation)

In [ ]:
visualisation(model,test)